<a href="https://colab.research.google.com/github/Sharanya-Parimanoharan/AI-Generated-Text-Detection/blob/main/DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import string
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve,roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

In [ ]:
# Load your dataset (data_2.csv)
df = pd.read_csv("drive/MyDrive/data_2.csv")
#dataset=load_dataset("artem9k/ai-text-detection-pile")
#df=pd.DataFrame(dataset['train'])
print(df.head())

In [ ]:

# Data preprocessing
# Remove punctuation
def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])

df['text']= df['text'].apply(remove_punctuation)

# Handle missing values
df.dropna(inplace=True)
df['text'].fillna("NA", inplace=True)

In [ ]:

# Convert labels to integers (assuming 'AI' is positive and others are negative)
df["label_int"] = np.where(df["label"] == '"AI"', 1, 0)


# split train dataset into train, validation and test sets
X_train, temp_text, y_train, temp_labels = train_test_split(df['text'], df['label_int'],
                                                                    random_state=2018,
                                                                    test_size=0.3,
                                                                    stratify=df['label_int'])


X_val, X_test, y_val, y_test = train_test_split(temp_text, temp_labels,
                                                                random_state=2018,
                                                                test_size=0.5,
                                                                stratify=temp_labels)

In [ ]:

# Load DistilBERT tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification

# Tokenize and preprocess the data
max_length = 128  # You can adjust this based on your dataset
batch_size = 32

In [ ]:

# Tokenize the training data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=max_length, return_tensors='pt', add_special_tokens=True)
train_labels = torch.tensor(list(y_train))
train_tokens = tokenizer.convert_ids_to_tokens(train_encodings["input_ids"][0])
print(train_tokens)

# Tokenize the testing data
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=max_length, return_tensors='pt', add_special_tokens=True)
test_labels = torch.tensor(list(y_test))

# Tokenize the val data
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, max_length=max_length, return_tensors='pt', add_special_tokens=True)
val_labels = torch.tensor(list(y_val))

# Create DataLoader for training and testing
train_data = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

val_data = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)


In [ ]:
# Set up optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 10  # You can adjust this based on your dataset


train_losses = []
val_losses = []

# Training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    print(f"Epoch [{epoch + 1}/{epochs}], Average training loss: {avg_train_loss:.4f}")

    # Validation loop
    model.eval()
    val_total_loss = 0
    for val_batch in val_dataloader:
        val_input_ids, val_attention_mask, val_labels = val_batch
        val_outputs = model(val_input_ids, attention_mask=val_attention_mask, labels=val_labels)
        val_loss = val_outputs.loss
        val_total_loss += val_loss.item()

    avg_val_loss = val_total_loss / len(val_dataloader)
    val_losses.append(avg_val_loss)
    print(f"Epoch [{epoch + 1}/{epochs}], Average validation loss: {avg_val_loss:.4f}")




In [ ]:
# plot training loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.show()

In [ ]:
# Evaluation
model.eval()
predicted_labels = []
true_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_label = np.argmax(logits, axis=1).tolist()
        true_label = labels.tolist()

        # Print some batch-wise information
        print("Input IDs:", input_ids)
        print("Logits:", logits)
        print("Predicted Label:", predicted_label)
        print("True Label:", true_label)

        predicted_labels.extend(predicted_label)
        true_labels.extend(true_label)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
report = classification_report(true_labels, predicted_labels)


print(true_labels)
print(predicted_labels)
print("Accuracy:", accuracy*100)
print("Classification Report:\n", report)


     # Calculate ROC curve
pr, tpr, thresholds = roc_curve(true_labels, predicted_labels)

    # Calculate AUC
auc_value = roc_auc_score(true_labels, predicted_labels)

       # Plot ROC curve
plt.figure()
plt.plot(pr, tpr, color='darkorange', lw=2, label=f'AUC = {auc_value:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Save the model
model.save_pretrained("classification_model")
tokenizer.save_pretrained("classification_model/tokenizer")

# Save model to Google Drive
model.save_pretrained("/content/drive/MyDrive/classification_model_withloss")
tokenizer.save_pretrained("/content/drive/MyDrive/classification_model_withloss/tokenizer")

In [ ]:
# Load the test dataset
test_df = pd.read_csv("drive/MyDrive/test_new.csv")

test_df['text'] = test_df['text'].apply(remove_punctuation)

# Convert labels to integers (assuming 'AI' is positive and others are negative)
test_df["label_int"] = np.where(test_df["label"] == '"AI"', 1, 0)


# Split the test dataset into features and labels
X_test = test_df["text"]
y_test = test_df["label_int"]


# Load the pre-trained model and tokenizer
model_name = "/content/drive/MyDrive/classification_model_withloss"
tokenizer_name = "/content/drive/MyDrive/classification_model_withloss/tokenizer"

loaded_model = DistilBertForSequenceClassification.from_pretrained(model_name)
#loaded_tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_name)

# Tokenize the testing data
max_length = 128  # You can adjust this based on your dataset
batch_size = 12

test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=max_length, return_tensors='pt', add_special_tokens=True)
test_labels = torch.tensor(list(y_test))

print(len(test_encodings['input_ids']))
print(len(test_encodings['attention_mask']))


# Create DataLoader for testing
test_data = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Evaluation on the test set
loaded_model.eval()
predicted_labels = []
true_labels = []


with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = loaded_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).tolist()
        true_label = labels.tolist()

        predicted_labels.extend(predicted_label)
        true_labels.extend(true_label)

np.savetxt('/content/drive/MyDrive/pred_DistilBERT.csv', predicted_labels, delimiter=',',  fmt='%.2f')


# Calculate metrics on the test set
accuracy = accuracy_score(true_labels, predicted_labels)
report = classification_report(true_labels, predicted_labels)

report = classification_report(true_labels, predicted_labels)
precision_ai = precision_score(true_labels, predicted_labels, pos_label=1)  # Assuming "AI" is the positive class
precision_human = precision_score(true_labels, predicted_labels, pos_label=0)  # Assuming "AI" is the positive class

recall_ai = recall_score(true_labels, predicted_labels, pos_label=1)  # Assuming "AI" is the positive class
recall_human = recall_score(true_labels, predicted_labels, pos_label=0)  # Assuming "AI" is the positive class

f1_ai = f1_score(true_labels, predicted_labels, pos_label=1)  # Assuming "AI" is the positive class
f1_human = f1_score(true_labels, predicted_labels, pos_label=0)  # Assuming "AI" is the positive class
conf_matrix = confusion_matrix(true_labels, predicted_labels)


print("Accuracy:", accuracy * 100)
print("Classification Report:\n", report)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data (replace with your actual values)
ai_values = [precision_ai,recall_ai,f1_ai]
human_values = [precision_human,recall_human,f1_human]

# Metrics names
metrics = [ 'Precision', 'Recall', 'F1-Score']

# Number of metrics
num_metrics = len(metrics)

# Create an array of indices for each metric
indices = np.arange(num_metrics)

# Bar width
bar_width = 0.1

# Plotting
fig, ax = plt.subplots(figsize=(6, 4))

# Plot AI values
ax.bar(indices, ai_values, bar_width,  color='blue',alpha=0.5)

# Plot Human values
ax.bar(indices + bar_width, human_values, bar_width, label='Human', color='orange')

# Customize the plot
ax.tick_params(axis='both', labelsize=15, labelcolor='black')

# Make the axis labels (x and y) bold
ax.xaxis.label.set_fontweight('bold')
ax.yaxis.label.set_fontweight('bold')
ax.set_xticks(indices + bar_width / 2)

ax.set_xticklabels(metrics)
ax.legend()
ax.set_ylabel('Metric Values',fontweight='bold')
#ax.set_title('Comparison of AI and Human Performance Metrics')

# Show the plot
plt.show()

In [ ]:
# Extract values from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Plotting
labels = ['True Negative', 'False Positive', 'False Negative', 'True Positive']
values = [tn, fp, fn, tp]

fig, ax = plt.subplots()
bars = ax.bar(labels, values, color=['pink', 'yellow', 'orange', 'blue'],alpha=0.8,width=0.3)

# Add labels and title
ax.set_ylabel('Count',fontweight='bold')
ax.set_title('Confusion Matrix')

# Add value annotations on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

ax.set_xticklabels(labels, fontweight='bold')

ax.tick_params(axis='y', labelsize=20, labelcolor='black')


plt.show()